In [ ]:
import numpy as np

In [ ]:
#入力データのロード
x = np.load('x.npy')
#教師用データのロード
t = np.load('t.npy')

In [ ]:
#tをN*3行列にする
t_bk=t
t=np.zeros((len(t),3))
for i in range(t.shape[0]):
    j=int(t_bk[i])
    t[i][j]=1.0

In [ ]:
#２層フィードフォワードニューラルネットワークモデル
def FNN(x,w,v,M,K):
    #x:入力データ  (N*D)→入力データはNセット分、まとめて一度に処理させる。入力データ１セットに対し出力値１セットを得る。
    #w:中間層への重み行列 (M*(D+1))
    #v:出力層への重み行列 (K*(M+1))
    #M:中間層の次元
    #K:出力層の次元
    N,D = x.shape
    b=np.zeros((N,M+1))  #中間層の入力総和
    z=np.zeros((N,M+1))  #中間層の出力
    a=np.zeros((N,K))    #出力層の入力総和
    y=np.zeros((N,K))    #出力層の出力
    
    #(1~n)個目の入力データセットについての処理
    for n in range(N):
        #中間層の計算
        for m in range(M):
            #入力データxを利用し、入力総和bのm番目の要素の算出を行う。結果はb[n,m]
            b[n,m]=np.dot(w[m,:],np.r_[x[n,:],1])
            #入力総和をシグモイド関数で出力値にする
            z[n,m]=sigmoid(b[n,m]) 

        #出力層の計算
        z[n,M] = 1  #最後のダミー用データ
        u=0
        for k in range(K):
            #中間層からの入力データzを利用し、入力総和aのk番目の要素の算出を行う。結果はa[n,k]
            a[n,k]=np.dot(v[k,:],z[n,:]) 
            #ソフトマックス関数用の入力総和
            u+=np.exp(a[n,k])
        for k in range(K):
            y[n,k]=np.exp(a[n,k])/u
    return y,a,z,b
            

In [ ]:
#誤差逆伝搬法
#∂E/∂w、∂E/∂vを求める
def back_propagation(w,v,M,K,x,t):
    N,D=x.shape
    y,a,z,b=FNN(x,w,v,M,K)
    dw=np.zeros((M,D+1))
    dv=np.zeros((K,M+1))
    delta1=np.zeros(M)
    delta2=np.zeros(K)
    
    for n in range(N):
        for k in range(K):
            delta2[k]=(y[n,k]-t[n,k])
        for j in range(M):
            delta1[j]=z[n,j]*(1-z[n,j])*np.dot(v[:,j],delta2)
        for k in range(K):
            dv[k,:]=dv[k,:]+delta2[k]*z[n,:]/N
        for j in range(M):
            dw[j,:]=dw[j,:]+delta1[j]*np.r_[x[n,:],1]/N
    return dw,dv
